In [1]:
import os, sys
import nest_asyncio
import asyncio

from fastapi import FastAPI
from fastapi.middleware.wsgi import WSGIMiddleware
import httpx

# 🧩 本地导入路径，根据放置的项目结构调整
sys.path.insert(0, r"E:\Github\a2a-samples\samples\python")

from components.api_key_dialog import api_key_dialog
from components.page_scaffold import page_scaffold
from pages.agent_list import agent_list_page
from pages.conversation import conversation_page
from pages.event_list import event_list_page
from pages.home import home_page_content
from pages.settings import settings_page_content
from pages.task_list import task_list_page
from service.server.server import ConversationServer
from state import host_agent_service
from state.state import AppState
import mesop as me
from contextlib import asynccontextmanager
from dotenv import load_dotenv

# ✅ 允许在 Notebook 中多次运行 asyncio
nest_asyncio.apply()

# ✅ 加载 .env 环境变量
load_dotenv()

# ⚙️ 初始化 FastAPI & 客户端封装
class HTTPXClientWrapper:
    async_client: httpx.AsyncClient = None

    def start(self):
        self.async_client = httpx.AsyncClient(timeout=30)

    async def stop(self):
        await self.async_client.aclose()
        self.async_client = None

    def __call__(self):
        assert self.async_client is not None
        return self.async_client


httpx_client_wrapper = HTTPXClientWrapper()

# ✅ 页面注册函数（照搬原代码结构）
def on_load(e: me.LoadEvent):
    state = me.state(AppState)
    me.set_theme_mode(state.theme_mode)
    if 'conversation_id' in me.query_params:
        state.current_conversation_id = me.query_params['conversation_id']
    else:
        state.current_conversation_id = ''
    uses_vertex_ai = os.getenv('GOOGLE_GENAI_USE_VERTEXAI', '').upper() == 'TRUE'
    api_key = os.getenv('GOOGLE_API_KEY', '')
    if uses_vertex_ai:
        state.uses_vertex_ai = True
    elif api_key:
        state.api_key = api_key
    else:
        state.api_key_dialog_open = True

security_policy = me.SecurityPolicy(allowed_script_srcs=['https://cdn.jsdelivr.net'])

@me.page(path='/', title='Chat', on_load=on_load, security_policy=security_policy)
def home_page():
    api_key_dialog()
    with page_scaffold():
        home_page_content(me.state(AppState))

@me.page(path='/agents', title='Agents', on_load=on_load, security_policy=security_policy)
def agents_page():
    api_key_dialog()
    agent_list_page(me.state(AppState))

@me.page(path='/conversation', title='Conversation', on_load=on_load, security_policy=security_policy)
def conversation():
    api_key_dialog()
    conversation_page(me.state(AppState))

@me.page(path='/event_list', title='Event List', on_load=on_load, security_policy=security_policy)
def events():
    api_key_dialog()
    event_list_page(me.state(AppState))

@me.page(path='/task_list', title='Task List', on_load=on_load, security_policy=security_policy)
def tasks():
    api_key_dialog()
    task_list_page(me.state(AppState))

@me.page(path='/settings', title='Settings', on_load=on_load, security_policy=security_policy)
def settings():
    api_key_dialog()
    settings_page_content()

# ✅ 定义 lifespan 生命周期（FastAPI 启动钩子）
@asynccontextmanager
async def lifespan(app: FastAPI):
    httpx_client_wrapper.start()
    ConversationServer(app, httpx_client_wrapper())
    app.openapi_schema = None
    app.mount("/", WSGIMiddleware(me.create_wsgi_app(debug_mode=False)))
    app.setup()
    yield
    await httpx_client_wrapper.stop()


In [2]:
import uvicorn
from fastapi import FastAPI

app = FastAPI(lifespan=lifespan)

# 环境变量设置（可手动设置）
host = os.environ.get('A2A_UI_HOST', '127.0.0.1')
port = int(os.environ.get('A2A_UI_PORT', '12000'))

host_agent_service.server_url = f"http://{host}:{port}"

# 启动服务
config = uvicorn.Config(app=app, host=host, port=port, timeout_graceful_shutdown=0)
server = uvicorn.Server(config)

await server.serve()


INFO:     Started server process [9180]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


INFO:     127.0.0.1:3427 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:3441 - "GET /styles.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:3442 - "GET /zone.js/bundles/zone.umd.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:3443 - "GET /prod_bundle.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:3451 - "POST /__ui__ HTTP/1.1" 200 OK
INFO:     127.0.0.1:3451 - "GET /__web-components-module__/components/async_poller.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:3470 - "POST /__ui__ HTTP/1.1" 200 OK
INFO:     127.0.0.1:3474 - "POST /conversation/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:3476 - "POST /task/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:3478 - "POST /message/pending HTTP/1.1" 200 OK
INFO:     127.0.0.1:3470 - "POST /__ui__ HTTP/1.1" 200 OK
INFO:     127.0.0.1:3509 - "POST /conversation/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:3518 - "POST /task/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:3523 - "POST /message/pending HTTP/1.1" 200 OK
INFO:     127.0.0.1:3555 - "POST /__ui__ HTTP/1.1" 200 OK
INFO:     127.0.

INFO:     Shutting down
ERROR:    Cancel 0 running task(s), timeout graceful shutdown exceeded
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [9180]
